**Visualization of the data**

1. Set up phase

In [ ]:
#Import of the needed libraries

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

# Opening JSON file
f = open('port_arthur.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
# Flatten each dictionary and store the results in a list
flattened_dicts = [flatten_dict(d) for d in data['data']]

# Create a DataFrame
data = pd.DataFrame(flattened_dicts)
 
# Closing file
f.close()

In [ ]:
#Modification of the variable naviguation_time in order to use it efficiently
data['navigation_time'] = pd.to_datetime(data['navigation_time'])

#Remove the duplicates
data = data.drop_duplicates()

data

In [ ]:
#Create a data frame only containing the tugs and another frame only containing the other vessels
tugs = data.loc[(data['vessel_type'] == 'tug')].reset_index()
non_tug = data.loc[(data['vessel_type'] != 'tug')].reset_index()

In [ ]:
# Select the columns we want to keep
selected_columns = ['vessel_name','vessel_type', 'device_dimensions_to_bow', 'device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port']

# Create a new DataFrame with only the selected columns
new_data = data.loc[:, selected_columns]

# Group the data by 'vessel_name' and aggregate the values (first function)
aggregated_df = new_data.groupby('vessel_name').first().reset_index()

# Display the aggregated DataFrame
aggregated_df

2. Visualization of the data

In [ ]:
# Define custom colors for each vessel type
custom_colors = ['mediumslateblue', 'goldenrod', 'darkgrey']

- Distribution of the vessel type within the data

In [ ]:
# Count the number of unique vessel types
vessel_type_counts = aggregated_df['vessel_type'].value_counts()

plt.figure(figsize=(8, 4))
plt.bar(vessel_type_counts.index, vessel_type_counts.values, color=custom_colors)
plt.xlabel('Vessel type')
plt.ylabel('Count')
plt.title('Distribution of vessel types')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Count the number of unique vessel types
vessel_type_counts = aggregated_df['vessel_type'].value_counts()

plt.figure(figsize=(7, 4))
plt.pie(vessel_type_counts, autopct='%1.1f%%', startangle=140, colors=custom_colors)
plt.title('Vessel type distribution (%)')

# Add a legend
plt.legend(vessel_type_counts.index, title='Vessel types', loc='best')
plt.axis('equal')  
plt.show()


- Box plots providing a visual summary of the distribution of the dimensions for all 3 different types of vessels

In [ ]:
# Create a 2x2 grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# List of dimensions
dimensions = ['device_dimensions_to_bow', 'device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port']

# Iterate over dimensions and corresponding subplots
for dimension, ax in zip(dimensions, axes.flatten()):
    sns.boxplot(data=aggregated_df, x='vessel_type', y=dimension, palette=custom_colors, ax=ax)
    ax.set_xlabel('Vessel type')
    ax.set_ylabel(f'Device dimensions to {dimension.split("_")[-1].title()}')
    
    # Calculate and display statistical properties
    for vessel_type in aggregated_df['vessel_type'].unique():
        subset_data = aggregated_df[aggregated_df['vessel_type'] == vessel_type]
        median = subset_data[dimension].median()
        q1 = subset_data[dimension].quantile(0.25)
        q3 = subset_data[dimension].quantile(0.75)
        max_val = subset_data[dimension].max()
        
        # Determine the vertical position for labels
        text_y = max_val + 2 if max_val + 2 < ax.get_ylim()[1] else ax.get_ylim()[1] - 2
        
        ax.text(
            subset_data.index[0], 
            text_y, 
            f"Median: {median:.2f}\nQ1: {q1:.2f}\nQ3: {q3:.2f}", 
            horizontalalignment='center', 
            verticalalignment='bottom',
            color='black'
        )

# Set a single title for all subplots
fig.suptitle('Dimensions', fontsize=16)
plt.tight_layout()
plt.show()

- Heatmap to visualize the correlations between device dimensions

In [ ]:

# List of device dimensions
device_dimensions = ['device_dimensions_to_bow', 'device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port']

correlation_matrix = aggregated_df[device_dimensions].corr()

# Create a figure with a larger size
plt.figure(figsize=(10, 8))


custom_cmap = sns.diverging_palette(220, 20, as_cmap=True)

# Create the correlation heatmap with annotations
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt='.2f', 
    cmap=custom_cmap,
    linewidths=0.5,
    square=True,  
    cbar=True,  
    cbar_kws={'shrink': 0.7},  
)

plt.title('Correlation heatmap of device dimensions')
plt.subplots_adjust(bottom=0.15)
plt.show()

- Pair plot with device dimensions to visually explore and understand (with a grid of scatter plots, histograms and kernel density estimates (KDEs)) the relationships and distributions between different device dimensions within the dataset

In [ ]:
# List of device dimensions
device_dimensions = ['device_dimensions_to_bow', 'device_dimensions_to_starboard', 'device_dimensions_to_stern', 'device_dimensions_to_port']

# Create a pair plot
pairplot = sns.pairplot(
    data=aggregated_df,
    vars=device_dimensions,
    hue='vessel_type',  
    palette=custom_colors,  
    diag_kind='kde', 
    plot_kws={'alpha': 0.7}, 
    diag_kws={'shade': True},  
)

# Add a title to the pair plot
pairplot.fig.suptitle('Pair plot of device dimensions', y=1.02)
plt.tight_layout()
plt.show()

- Navigation speed over time for each tug

In [ ]:
# Iterate through unique vessel names
unique_vessel_names = tugs['vessel_name'].unique()

plt.figure(figsize=(12, 8))

for vessel_name in unique_vessel_names:
    # Filter data for the current vessel name
    subset = tugs[tugs['vessel_name'] == vessel_name]
    
    # Create a line plot for navigation_speed over time for the current vessel
    plt.plot(subset['navigation_time'], subset['navigation_speed'], label=vessel_name)

# Set plot labels and title
plt.xlabel('Time')
plt.ylabel('Navigation speed')
plt.title('Navigation speed over time for each tug')
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  
plt.show()

- Navigation speed over time for each tug, with the possibility here to get the values on the curves

In [ ]:
# Filter data for non-tug vessels
tug_data = data[data['vessel_type'] == 'tug']

# Create a figure
fig = go.Figure()

# Iterate through unique vessel names for non-tug vessels
unique_tug_names = tug_data['vessel_name'].unique()

for vessel_name in unique_tug_names:
    # Filter data for the current vessel name
    subset = tug_data[tug_data['vessel_name'] == vessel_name]
    
    # Create a line trace for navigation_speed over time for the current vessel
    fig.add_trace(go.Scatter(x=subset['navigation_time'], y=subset['navigation_speed'],
                             mode='lines+markers', name=vessel_name,
                             hoverinfo='x+y', text=subset['navigation_time'].dt.strftime('%Y-%m-%d %H:%M:%S')))

# Customize layout
fig.update_layout(
    title='Navigation speed over time for each tug',
    xaxis_title='Time',
    yaxis_title='Navigation speed',
    xaxis=dict(tickangle=-45),
    legend=dict(x=1, y=1)
)

# Show the plot
fig.show()

- Navigation speed over time for each vessel (non-tug)

In [ ]:
# Iterate through unique vessel names
unique_vessel_names = non_tug['vessel_name'].unique()

plt.figure(figsize=(12, 8))

for vessel_name in unique_vessel_names:
    # Filter data for the current vessel name
    subset = non_tug[non_tug['vessel_name'] == vessel_name]
    
    # Create a line plot for navigation_speed over time for the current vessel
    plt.plot(subset['navigation_time'], subset['navigation_speed'], label=vessel_name)

# Set plot labels and title
plt.xlabel('Time')
plt.ylabel('Navigation speed')
plt.title('Navigation speed over time for each vessel (non-tug vessels)')
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))  # Add legend outside the plot

plt.show()

- Navigation speed over time for each vessel (no tug), with the possibility here to get the values on the curves

In [ ]:
# Filter data for non-tug vessels
non_tug_data = data[data['vessel_type'] != 'tug']

# Create a figure
fig = go.Figure()

# Iterate through unique vessel names for non-tug vessels
unique_vessel_names = non_tug_data['vessel_name'].unique()

for vessel_name in unique_vessel_names:
    # Filter data for the current vessel name
    subset = non_tug_data[non_tug_data['vessel_name'] == vessel_name]
    
    # Create a line trace for navigation_speed over time for the current vessel
    fig.add_trace(go.Scatter(x=subset['navigation_time'], y=subset['navigation_speed'],
                             mode='lines+markers', name=vessel_name,
                             hoverinfo='x+y', text=subset['navigation_time'].dt.strftime('%Y-%m-%d %H:%M:%S')))

# Customize layout
fig.update_layout(
    title='Navigation speed over time for each vessel (non-tug vessels)',
    xaxis_title='Time',
    yaxis_title='Navigation speed',
    xaxis=dict(tickangle=-45),
    legend=dict(x=1, y=1)
)

fig.show()


- Navigation speed over time, possibility to choose 2 desired vessels in the top bars to compare their speed (for example a tug and a tanker to see if the ^y had to same speed during a period of time)

In [ ]:
# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(id='vessel-selector-1', multi=False, placeholder="Select Vessel 1"),
    dcc.Dropdown(id='vessel-selector-2', multi=False, placeholder="Select Vessel 2"),
    dcc.Graph(id='speed-comparison'),
])

# Create a color map to assign unique colors to vessels
vessel_colors = px.colors.qualitative.Set3

# Populate the dropdown menus with vessel names
vessel_dropdown = [{"label": vessel_name, "value": vessel_name} for vessel_name in data['vessel_name'].unique()]
@app.callback(
    Output('vessel-selector-1', 'options'),
    Output('vessel-selector-2', 'options'),
    Input('vessel-selector-1', 'value'),
    Input('vessel-selector-2', 'value')
)
def update_vessel_options(selected_vessel_1, selected_vessel_2):
    # Exclude the selected vessel from the other dropdown
    available_vessels = [vessel for vessel in data['vessel_name'].unique() if vessel != selected_vessel_1]
    
    return ([{"label": vessel, "value": vessel} for vessel in available_vessels],
            [{"label": vessel, "value": vessel} for vessel in available_vessels])

# Define callback function to update speed comparison graph
@app.callback(
    Output('speed-comparison', 'figure'),
    Input('vessel-selector-1', 'value'),
    Input('vessel-selector-2', 'value')
)
def update_speed_comparison(selected_vessel_1, selected_vessel_2):
    if selected_vessel_1 is None or selected_vessel_2 is None:
        return go.Figure()
    
    # Filter data for the selected vessels
    selected_vessel_data_1 = data[data['vessel_name'] == selected_vessel_1]
    selected_vessel_data_2 = data[data['vessel_name'] == selected_vessel_2]
    
    # Assign unique colors to the selected vessels
    color_1 = vessel_colors[0]
    color_2 = vessel_colors[1]
    
    # Create line traces for the selected vessels
    trace_1 = go.Scatter(x=selected_vessel_data_1['navigation_time'], y=selected_vessel_data_1['navigation_speed'],
                         mode='lines+markers', hoverinfo='x+y',
                         text=selected_vessel_data_1['navigation_time'].dt.strftime('%Y-%m-%d %H:%M:%S'),
                         line=dict(color=color_1), name=selected_vessel_1)
    trace_2 = go.Scatter(x=selected_vessel_data_2['navigation_time'], y=selected_vessel_data_2['navigation_speed'],
                         mode='lines+markers', hoverinfo='x+y',
                         text=selected_vessel_data_2['navigation_time'].dt.strftime('%Y-%m-%d %H:%M:%S'),
                         line=dict(color=color_2), name=selected_vessel_2)
    
    # Create the speed comparison graph
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Speed Comparison",))
    fig.add_trace(trace_1)
    fig.add_trace(trace_2)
    fig.update_layout(
        title_text=f'Speed comparison between {selected_vessel_1} and {selected_vessel_2}',
        xaxis_title='Time',
        yaxis_title='Navigation speed',
        xaxis=dict(tickangle=-45),
    )
    
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

- Pie charts of the navigation_status

In [ ]:
# Calculate the counts of each navigation status
navigation_status_counts = tugs['navigation_status'].value_counts()

plt.figure(figsize=(10, 4))

# Create the pie chart
plt.pie(navigation_status_counts, autopct='%1.1f%%', startangle=140)

# Add a legend with labels
plt.legend(navigation_status_counts.index, title='Navigation status of tugs', loc='upper right')

plt.title('Distribution of tugs navigation status')
plt.axis('equal')
plt.show()

In [ ]:
# Pie chart of navigation_status
# Calculate the counts of each navigation status
navigation_status_counts = non_tug['navigation_status'].value_counts()

plt.figure(figsize=(12, 6))

# Create the pie chart
plt.pie(navigation_status_counts, autopct='%1.1f%%', startangle=140)

# Add a legend with labels
plt.legend(navigation_status_counts.index, title='Navigation status of the vessels', loc='upper right')

plt.title('Distribution of vessels navigation status')
plt.axis('equal')
plt.show()

In [ ]:
# Group data by 'navigation_status' and 'vessel_type' and count occurrences
status_type_counts = data.groupby(['navigation_status', 'vessel_type']).size().unstack(fill_value=0)

# Create a stacked bar chart with custom colors
ax = status_type_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=custom_colors)

# Add values on top of the bars
for container in ax.containers:
    ax.bar_label(container, label_type='edge', fontsize=10, color='black', labels=container.datavalues)

plt.xlabel('Navigation status')
plt.ylabel('Count')
plt.title('Stacked bar chart of vessel type within navigation status')

# Customize legend
ax.legend(title='Vessel Type', loc='upper left')

plt.show()
